# Exercise 5: The Agentic Loop

In this exercise, we put it all together into an **agent loop** — the core pattern behind AI agents.

By the end, you'll understand:
- How to build a loop that keeps running until the model stops requesting tools
- How to dispatch tool calls to actual functions
- How the model can chain multiple tool calls to answer a single question

## Step 1: Setup

In [1]:
import anthropic

client = anthropic.Anthropic()

## Step 2: Define tools and a dispatcher

Instead of hardcoding tool results inline, we create a `call_tool` function that maps tool names to their implementations. This is how real agents work — the dispatcher is the bridge between the model and your code.

In [2]:
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                }
            },
            "required": ["location"],
        },
    }
]

def call_tool(tool_name, tool_input):
    if tool_name == "get_weather":
        return "65 degrees"
    else:
        return "Tool not found."

## Step 3: The agent loop

This is the core pattern. The loop:
1. Sends messages to Claude
2. Checks if the response contains any `tool_use` blocks
3. If yes — executes each tool, collects results, appends them, and loops back to step 1
4. If no — the model is done, we print the final text and break

In [3]:
messages = [{"role": "user", "content": "What is the weather in Tokyo?"}]

loop_count = 0

while True:
    loop_count += 1
    print(f"--- Loop iteration {loop_count} ---")

    message = client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=1024,
        messages=messages,
        tools=tools,
    )

    print(f"stop_reason: {message.stop_reason}")

    # Add assistant response to messages
    messages.append({"role": "assistant", "content": message.content})

    # Collect tool results
    user_message = {"role": "user", "content": []}
    has_tool_use = False

    for content in message.content:
        if content.type == "tool_use":
            has_tool_use = True
            print(f"  Calling tool: {content.name}({content.input})")
            tool_result = call_tool(content.name, content.input)
            print(f"  Result: {tool_result}")
            user_message["content"].append(
                {
                    "type": "tool_result",
                    "tool_use_id": content.id,
                    "content": tool_result,
                }
            )
        elif content.type == "text":
            print(f"  Text: {content.text}")

    if has_tool_use:
        messages.append(user_message)
    else:
        print("\nDone! No more tool calls.")
        break

--- Loop iteration 1 ---
stop_reason: tool_use
  Calling tool: get_weather({'location': 'Tokyo, Japan'})
  Result: 65 degrees
--- Loop iteration 2 ---
stop_reason: end_turn
  Text: The current weather in Tokyo is **65 degrees Fahrenheit**.

Done! No more tool calls.


## Step 4: Review the full message history

In [4]:
for msg in messages:
    print(f"--- {msg['role']} ---")
    print(f"{msg['content']}\n")

--- user ---
What is the weather in Tokyo?

--- assistant ---
[ToolUseBlock(id='toolu_017h7hxT9FvGSawiT3SPQMEo', input={'location': 'Tokyo, Japan'}, name='get_weather', type='tool_use', caller={'type': 'direct'})]

--- user ---
[{'type': 'tool_result', 'tool_use_id': 'toolu_017h7hxT9FvGSawiT3SPQMEo', 'content': '65 degrees'}]

--- assistant ---
[TextBlock(citations=None, text='The current weather in Tokyo is **65 degrees Fahrenheit**.', type='text')]



## Key takeaway

This is the **agentic loop** — the foundation of every AI agent:

```
while True:
    response = call_model(messages)
    if response has tool calls:
        execute tools, append results
    else:
        break  # model is done
```

Everything else — multiple tools, error handling, human-in-the-loop, memory — is built on top of this pattern.